In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
import zarr
import glob
import s3fs
import json
import fsspec

In [2]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster

In [3]:
cluster = KubeCluster()
cluster.adapt(minimum=4, maximum=20, interval='2s', wait_count=3)
client = Client(cluster)
cluster

In [73]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True)

CPU times: user 7.21 s, sys: 299 ms, total: 7.51 s
Wall time: 42.8 s


In [61]:
ds_sst

,Array,Chunk
Bytes,16.70 TB,129.53 MB
Shape,"(6443, 17999, 36000)","(5, 1799, 3600)"
Count,141791 Tasks,141790 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,16.70 TB,129.53 MB
Shape,"(6443, 17999, 36000)","(5, 1799, 3600)"
Count,141791 Tasks,141790 Chunks
Type,float32,numpy.ndarray


In [6]:
%%time
s3 = s3fs.S3FileSystem(
    anon=False,
    key='xxx',    
    secret='xxx',
    client_kwargs=dict(region_name='us-west-2')
)

CPU times: user 32.5 ms, sys: 0 ns, total: 32.5 ms
Wall time: 31.5 ms


# First Chunks - 60x1799x3600

In [7]:
ds_subset = ds_sst.sel(time=slice('2002-06-01', '2002-07-30'))
ds_subset.analysed_sst.encoding = {}
ds_subset.analysis_error.encoding = {}
ds_subset.mask.encoding = {}
ds_subset.sea_ice_fraction.encoding = {}
ds_subset

,Array,Chunk
Bytes,77.76 GB,64.76 MB
Shape,"(60, 17999, 36000)","(5, 1799, 3600)"
Count,143111 Tasks,1320 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,77.76 GB,64.76 MB
Shape,"(60, 17999, 36000)","(5, 1799, 3600)"
Count,143111 Tasks,1320 Chunks
Type,int16,numpy.ndarray
,Array,Chunk


## Creating the initial store

In [8]:
%%time

chunks = {'time': 60, 'lat': 1799, 'lon': 3600}

chunk_string = '_'.join(map(str, chunks.values()))

ds_rechunked = ds_subset.chunk(chunks)
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'writing to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'w'}
ds_rechunked.to_zarr(s3store, **args)

writing to mur-sst/zarr-60_1799_3600
CPU times: user 34.8 s, sys: 1.27 s, total: 36 s
Wall time: 4min 11s


In [9]:
%%time

file_location = f's3://mur-sst/zarr-{chunk_string}'
ds_rechunked = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True, mask_and_scale=False)
ds_rechunked

CPU times: user 83.5 ms, sys: 10.5 ms, total: 94 ms
Wall time: 380 ms


<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 60)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2002-06-01T09:00:00 ... 2002-07-30T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) int16 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) int16 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) int8 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) int8 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

## Appending

In [74]:
ds_subset = ds_sst.sel(time=slice('2002-09-29', '2002-11-27'))
ds_subset.analysed_sst.encoding = {}
ds_subset.analysis_error.encoding = {}
ds_subset.mask.encoding = {}
ds_subset.sea_ice_fraction.encoding = {}
ds_subset

,Array,Chunk
Bytes,155.51 GB,129.53 MB
Shape,"(60, 17999, 36000)","(5, 1799, 3600)"
Count,143111 Tasks,1320 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,155.51 GB,129.53 MB
Shape,"(60, 17999, 36000)","(5, 1799, 3600)"
Count,143111 Tasks,1320 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [77]:
# check store
chunks = {'time': 60, 'lat': 1799, 'lon': 3600}
file_location = f's3://mur-sst/zarr-{chunk_string}'
ds_rechunked = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True, mask_and_scale=False)
ds_rechunked

<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 180)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2002-06-01T09:00:00 ... 2002-11-27T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) int16 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) int16 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) int8 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) int8 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [76]:
%%time
ds_rechunked = ds_subset.chunk(chunks)
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'appending to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'a', 'append_dim': 'time'}
ds_rechunked.to_zarr(s3store, **args)

appending to mur-sst/zarr-60_1799_3600
CPU times: user 32.8 s, sys: 1.06 s, total: 33.9 s
Wall time: 2min 36s


# Performance + Integrity Testing

In [78]:
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True, mask_and_scale=False)
ds_subset = ds_sst.sel(time=slice('2002-06-01', '2002-11-27'))
ds_subset

,Array,Chunk
Bytes,233.27 GB,64.76 MB
Shape,"(180, 17999, 36000)","(5, 1799, 3600)"
Count,145751 Tasks,3960 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,233.27 GB,64.76 MB
Shape,"(180, 17999, 36000)","(5, 1799, 3600)"
Count,145751 Tasks,3960 Chunks
Type,int16,numpy.ndarray
,Array,Chunk


In [79]:
%%time
print(ds_subset.analysed_sst.sel(time=slice('2002-10-02','2002-11-10')).mean().values)
# print()
# print(ds_subset.analysed_sst.sel(time='2003-03-01',lat=slice(20,50)).mean().values)

-18520.321555898976
CPU times: user 4.32 s, sys: 221 ms, total: 4.54 s
Wall time: 20.1 s


In [80]:
%%time
print(ds_rechunked.analysed_sst.sel(time=slice('2002-10-02','2002-11-10')).mean().values)
# print()
# print(ds_rechunked.analysed_sst.sel(time='2003-03-01',lat=slice(20,50)).mean().values)

-18520.321555898976
CPU times: user 1.68 s, sys: 111 ms, total: 1.8 s
Wall time: 33.1 s
